# Import

In [0]:
import pandas as pd
import pyspark.sql.functions as sf
from pyspark.sql import SparkSession
from functions import *
from pyspark.sql.types import IntegerType

# Extract 

In [0]:
# Government expenditure on education, total (% of GDP) datainnhenting 
expenditure_education_data = pd.read_csv(
    '/Volumes/dev_truls/default/data_files/API_SE.XPD.TOTL.GD.ZS_DS2_en_csv_v2_391679.csv'
)
display(expenditure_education_data)

In [0]:
# Expense (% of GDP) datainnhenting
expense_gpd_data = pd.read_csv('/Volumes/dev_truls/default/data_files/API_GC.XPN.TOTL.GD.ZS_DS2_en_csv_v2_86912.csv')
display(expense_gpd_data)

In [0]:
# GPD per capita datainnhenting
gpd_per_capita_data = pd.read_csv('/Volumes/dev_truls/default/data_files/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_394227.csv')
display(gpd_per_capita_data)

In [0]:
# Life expectancy at birth, total (years) datainnhenting
life_expectancy_at_birth_data = pd.read_csv('/Volumes/dev_truls/default/data_files/API_SP.DYN.LE00.IN_DS2_en_csv_v2_393864.csv')

display(life_expectancy_at_birth_data)

# Transformation

In [0]:
# Bruker melt for å endre strukturen fra wide format til long format for alle de fire indikator-dataene.
# Dette gjør det enklere å analysere, sammenligne og slå sammen indikatorene senere i prosessen.
expenditure_education_data = pd.melt(expenditure_education_data, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name='Year', value_name='Value')

expense_gpd_data = pd.melt(expense_gpd_data, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name='Year', value_name='Value')

gpd_per_capita_data = pd.melt(gpd_per_capita_data, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name='Year', value_name='Value')

life_expectancy_at_birth_data = pd.melt(life_expectancy_at_birth_data, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name='Year', value_name='Value')

In [0]:
# Konverter datsetettet fra pandas dataframe til spark dataframe for å kunne bruke Spark SQL og utnytte distributert databehandling.
spark = SparkSession.builder.getOrCreate()
expenditure_education_df = spark.createDataFrame(expenditure_education_data)
expense_gpd_df = spark.createDataFrame(expense_gpd_data)
gpd_per_capita_df = spark.createDataFrame(gpd_per_capita_data)
life_expectancy_at_birth_df = spark.createDataFrame(life_expectancy_at_birth_data)

In [0]:
# Bruker unionByName for å sammenstille alle 4 indikator-dataframene til en Spark dataframe for mer effektiv analysering og lagring etter melt().
# Selv om melt gir samme kolonnenavn og struktur, sikrer unionByName at DataFramene slås sammen korrekt basert på kolonnenmavn,
# og unnår feil dersom rekkefølgen på kolonnene skulle være ulik eller endres senere.
# Dette gir en mer robust og fremtidssikker løsning enn vanlig union

all_indicators_df = expenditure_education_df.unionByName(expense_gpd_df).unionByName(gpd_per_capita_df).unionByName(life_expectancy_at_birth_df)

In [0]:
# Filtrerer dataene automatisk til kun å inkludere de siste 15 årene basert på inneværende år.
# Dette reduserer antall rader ved å fjerne alle observasjoner eldre enn 15 år.
current_year = pd.Timestamp.now().year
all_indicators_df = all_indicators_df[all_indicators_df['Year'] >= str(current_year - 15)]

In [0]:
all_indicators_df = all_indicators_df.withColumn(
    "Status",
    sf.when(sf.col("Value").isNull(), "Not reported / Not collected").otherwise("Reported")
)

In [0]:
all_indicators_df = all_indicators_df.withColumnRenamed(
'Country Name', 'Country')

In [0]:
all_indicators_df = rename_columns_to_snake_case(all_indicators_df)

In [0]:
all_indicators_df = all_indicators_df.withColumn('Year', all_indicators_df['Year'].cast(IntegerType()))

In [0]:
display(all_indicators_df)

# Write